# Sentiment analysis

In [1]:
from __future__ import division

from afinn import Afinn
import numpy as np
import pandas as pd

In [2]:
# http://www.crowdflower.com/data-for-everyone
url = 'https://www.crowdflower.com/wp-content/uploads/2016/03/Twitter-sentiment-self-drive-DFE.csv'
df = pd.read_csv(url)

In [3]:
# Sentiment analysis with AFINN
afinn = Afinn()
afinn_scores = [afinn.score(text) for text in df.text]
df['afinn'] = afinn_scores
df[['sentiment', 'afinn', 'text']].head(10)

,sentiment,afinn,text
0,5,0.0,Two places I'd invest all my money if I could:...
1,5,5.0,Awesome! Google driverless cars will help the ...
2,2,1.0,If Google maps can't keep up with road constru...
3,2,0.0,Autonomous cars seem way overhyped given the t...
4,3,0.0,Just saw Google self-driving car on I-34. It w...
5,3,0.0,Will driverless cars eventually replace taxi d...
6,not_relevant,0.0,Chicago metro expected to be fully autonomous ...
7,not_relevant,3.0,I love the infotainment system in my new car. ...
8,5,-3.0,Autonomous vehicles could reduce traffic fatal...
9,1,1.0,Driverless cars are not worth the risk. Don't...


In [4]:
pd.crosstab(df.sentiment, df.afinn)

afinn,-11.0,-10.0,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,...,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,12.0
sentiment,,,,,,,,,,,,,,,,,,,,,
1,1,1,0,1,4,5,2,13,16,12,...,6,5,1,0,2,0,0,0,0,0
2,0,1,2,4,7,7,10,33,54,115,...,66,38,21,4,6,1,0,0,1,0
3,0,0,0,1,3,5,14,38,85,212,...,318,157,83,24,10,8,5,0,2,0
4,1,0,0,1,3,7,10,24,40,87,...,177,151,90,34,22,7,9,1,0,0
5,0,0,1,0,1,0,0,5,14,14,...,55,70,61,13,14,4,5,2,1,1
not_relevant,0,0,1,0,1,2,2,4,2,9,...,22,15,12,4,4,2,0,0,1,0


In [26]:
# Ignore tweets that are marked as 'not relevant'
df_relevant = df.loc[df.sentiment != 'not_relevant', :]
df_relevant.sentiment = pd.to_numeric(df_relevant.sentiment)

In [6]:
# Compute 3-class confusion matrix
confusion = pd.crosstab(np.sign(df_relevant.sentiment - 3), np.sign(df_relevant.afinn))
confusion

afinn,-1.0,0.0,1.0
sentiment,,,
-1,357,213,225
0,602,2696,947
1,304,617,982


In [7]:
accuracy_3_class = np.sum(np.diag(confusion)) / np.sum(confusion.values)
accuracy_3_class

0.58116088146334433

In [8]:
# Compute 2-class confusion matrix
confusion_2_class = confusion.iloc[[0, 2], [0, 2]]
confusion_2_class

afinn,-1.0,1.0
sentiment,,
-1,357,225
1,304,982


In [9]:
accuracy_2_class = np.sum(np.diag(confusion_2_class)) / np.sum(confusion_2_class.values)
accuracy_2_class

0.71680942184154173

In [12]:
accuracy_2_class_baseline = confusion_2_class.sum().max() / np.sum(confusion_2_class.values)
accuracy_2_class_baseline 

0.64614561027837258